In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install smplx
!pip install opencv-python
!pip install trimesh
!pip install pyrender
!pip install matplotlib
!pip install pyopengl
!pip install mediapipe
!pip install pyopengl pyopengl_accelerate


Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.


  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [1]:
import torch
import smplx
import numpy as np
import trimesh
import pyrender

# ========== Load SMPL model ==========
MODEL_PATH = "./SMPL_models"   # <-- replace with the folder containing smpl.pkl
smpl = smplx.SMPL(model_path=MODEL_PATH, gender="NEUTRAL").to("cpu")

# Shape and pose parameters
betas = torch.zeros([1, 10], device="cpu")
body_pose = torch.zeros([1, 69], device="cpu")

output = smpl(betas=betas, body_pose=body_pose)
vertices = output.vertices[0].detach().cpu().numpy()
faces = smpl.faces

mesh = trimesh.Trimesh(vertices, faces, process=False)
mesh_pr = pyrender.Mesh.from_trimesh(mesh, smooth=True)

# ========== Scene ==========
scene = pyrender.Scene()
scene.add(mesh_pr)

# Add camera
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
camera_pose = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, -0.5],   # move up a bit
    [0, 0, 1, 2.5],    # move back a bit
    [0, 0, 0, 1]
])
scene.add(camera, pose=camera_pose)

# Add light
light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
scene.add(light, pose=np.eye(4))

# ========== Viewer ==========
print("Launching pyrender viewer... (press ESC to quit)")
pyrender.Viewer(scene, use_raymond_lighting=True, viewport_size=(800, 800))


Launching pyrender viewer... (press ESC to quit)


Viewer=(width=800, height=800)

In [2]:
import torch
import numpy as np
import trimesh
import pyrender
import cv2

from smplx import SMPL

# -------------------------------
# 1. Load SMPL model (T-pose)
# -------------------------------
smpl_model_path = "./smpl_models"  # adjust to your path
smpl = SMPL(model_path=smpl_model_path, gender='NEUTRAL')

# Zero pose & shape → T-pose
betas = torch.zeros([1, 10], dtype=torch.float32)
body_pose = torch.zeros([1, 69], device="cpu")

output = smpl(betas=betas, body_pose=body_pose)
vertices = output.vertices[0].detach().cpu().numpy()
faces = smpl.faces

# -------------------------------
# 2. Render SMPL (offscreen)
# -------------------------------
mesh = trimesh.Trimesh(vertices, faces, process=False)
pyr_mesh = pyrender.Mesh.from_trimesh(mesh, smooth=True)

scene = pyrender.Scene(bg_color=[255, 255, 255, 255], ambient_light=[0.8, 0.8, 0.8])
scene.add(pyr_mesh)

# ---- Add Camera ----
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
cam_pose = np.array([
    [1.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, -0.5],   # move camera up a bit
    [0.0, 0.0, 1.0, 2.5],    # move camera away from SMPL
    [0.0, 0.0, 0.0, 1.0]
])
scene.add(camera, pose=cam_pose)

# ---- Add Light ----
light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
scene.add(light, pose=cam_pose)

# ---- Offscreen Renderer ----
renderer = pyrender.OffscreenRenderer(viewport_width=512, viewport_height=512)
color, depth = renderer.render(scene)
renderer.delete()

# Convert to OpenCV format
smpl_img = cv2.cvtColor(color, cv2.COLOR_RGBA2BGRA)

# -------------------------------
# 3. Load 2D shirt
# -------------------------------
shirt_img = cv2.imread("./database/shirt2.png", cv2.IMREAD_UNCHANGED)

# -------------------------------
# 4. Define torso region (approx)
# -------------------------------
shift_up = 88
h, w, _ = smpl_img.shape
x1, y1 = int(w*0.3), int(h*0.35) - shift_up  # left shoulder
x2, y2 = int(w*0.7), int(h*0.65) - shift_up  # right hip

sh_h, sh_w = shirt_img.shape[:2]
torso_width  = (x2 - x1) 
torso_height = (y2 - y1)
scale = min(torso_width / sh_w, torso_height / sh_h)
old_w = int(sh_w * scale)
old_h = int(sh_h * scale)

shrink_factor = 0.72
new_w = int(old_w * shrink_factor)
new_h = int(old_h * shrink_factor)


resized_shirt = cv2.resize(shirt_img, (new_w, new_h), interpolation=cv2.INTER_AREA)

# -------------------------------
# 5. Overlay shirt on SMPL
# -------------------------------
offset_x = x1 + (torso_width - new_w) // 2
offset_y = y1 + (torso_height - new_h) // 2

overlay = smpl_img.copy()
for y in range(new_h):
    for x in range(new_w):
        if resized_shirt[y, x, 3] > 0:  # alpha
            overlay[offset_y+y, offset_x+x, :3] = resized_shirt[y, x, :3]

# -------------------------------
# 3b. Load 2D pants
# -------------------------------
pant_img = cv2.imread("./database/cargo.png", cv2.IMREAD_UNCHANGED)

# -------------------------------
# 4b. Define hip/leg region (approx)
# -------------------------------
shift_up = 95
x1_p, y1_p = int(w*0.35), int(h*0.55) - shift_up # left hip
x2_p, y2_p = int(w*0.65), int(h*0.85) - shift_up # right thigh

pant_h, pant_w = pant_img.shape[:2]
pant_width  = (x2_p - x1_p)
pant_height = (y2_p - y1_p)

# Make pant a bit wider
scale_w = 1.20 * (pant_width / pant_w)  # 20% wider
scale_h = pant_height / pant_h         # keep vertical scale the same

new_w_p = int(pant_w * scale_w)
new_h_p = int(pant_h * scale_h)

resized_pant = cv2.resize(pant_img, (new_w_p, new_h_p), interpolation=cv2.INTER_AREA)

    
# shrink_factor = 1.00
# new_w_p = int(old_w_p * shrink_factor)
# new_h_p = int(old_h_p * shrink_factor)

# resized_pant = cv2.resize(pant_img, (new_w_p, new_h_p), interpolation=cv2.INTER_AREA)

# -------------------------------
# 5b. Overlay pants on SMPL
# -------------------------------
offset_x_p = x1_p + (pant_width - new_w_p) // 2 
offset_y_p = y1_p + (pant_height - new_h_p) // 2 + 25

for y in range(new_h_p):
    for x in range(new_w_p):
        if resized_pant[y, x, 3] > 0:  # alpha
            overlay[offset_y_p+y, offset_x_p+x, :3] = resized_pant[y, x, :3]

# -------------------------------
# 6b. Show result
# -------------------------------
cv2.imshow("SMPL + Shirt + Pants", overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [4]:
import torch
import numpy as np
import trimesh
import pyrender
import cv2
from smplx import SMPL

# -------------------------------
# 1. Load SMPL model (T-pose)
# -------------------------------
smpl_model_path = "./smpl_models"  # adjust to your path
smpl = SMPL(model_path=smpl_model_path, gender='NEUTRAL')

betas = torch.zeros([1, 10], dtype=torch.float32)
body_pose = torch.zeros([1, 69], device="cpu")

output = smpl(betas=betas, body_pose=body_pose)
vertices = output.vertices[0].detach().cpu().numpy()
faces = smpl.faces
joints = output.joints[0].detach().cpu().numpy()

# -------------------------------
# 2. Render SMPL (offscreen)
# -------------------------------
mesh = trimesh.Trimesh(vertices, faces, process=False)
pyr_mesh = pyrender.Mesh.from_trimesh(mesh, smooth=True)

scene = pyrender.Scene(bg_color=[255, 255, 255, 255], ambient_light=[0.8, 0.8, 0.8])
scene.add(pyr_mesh)

camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
cam_pose = np.array([
    [1.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, -0.5],
    [0.0, 0.0, 1.0, 2.5],
    [0.0, 0.0, 0.0, 1.0]
])
scene.add(camera, pose=cam_pose)

light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
scene.add(light, pose=cam_pose)

renderer = pyrender.OffscreenRenderer(viewport_width=512, viewport_height=512)
color, depth = renderer.render(scene)
renderer.delete()

# Convert to OpenCV BGRA
smpl_img = cv2.cvtColor(color, cv2.COLOR_RGBA2BGRA)
H, W = smpl_img.shape[:2]

# -------------------------------
# Helper: Project 3D → 2D
# -------------------------------
def project_to_image(point_3d, camera_pose, camera, W, H):
    point_h = np.append(point_3d, 1)
    cam_point = np.linalg.inv(camera_pose) @ point_h
    x = cam_point[0] / cam_point[2]
    y = cam_point[1] / cam_point[2]
    f = 0.5 * H / np.tan(camera.yfov / 2)
    u = int(W/2 + x*f)
    v = int(H/2 - y*f)
    return u, v

def clamp(val, min_val, max_val):
    return max(min_val, min(val, max_val))

def overlay_rgba(background, overlay_img, x, y):
    bh, bw = background.shape[:2]
    oh, ow = overlay_img.shape[:2]
    result = background.copy()

    for i in range(oh):
        for j in range(ow):
            yy, xx = y + i, x + j
            if 0 <= yy < bh and 0 <= xx < bw:
                if overlay_img.shape[2] == 4:
                    alpha = overlay_img[i, j, 3] / 255.0
                else:
                    alpha = 1.0
                if alpha > 0:
                    result[yy, xx, :3] = (
                        alpha * overlay_img[i, j, :3] +
                        (1 - alpha) * result[yy, xx, :3]
                    )
    return result

# -------------------------------
# 3. Load 2D shirt & pants
# -------------------------------
shirt_img = cv2.imread("./database/shirt2.png", cv2.IMREAD_UNCHANGED)
pant_img  = cv2.imread("./database/cargo.png", cv2.IMREAD_UNCHANGED)

# -------------------------------
# 4. Get SMPL joints → 2D
# -------------------------------
joint_indices = {
    "left_shoulder": 30,
    "right_shoulder": 33,
    "left_hip": 4,
    "right_hip": 5,
    "left_knee": 1,
    "right_knee": 2,
    "left_toe": 13,
    "right_toe": 14
}

joint_2d = {}
for name, idx in joint_indices.items():
    u, v = project_to_image(joints[idx], cam_pose, camera, W, H)
    joint_2d[name] = (clamp(u, 0, W-1), clamp(v, 0, H-1))

# -------------------------------
# 5. Auto-fit shirt
# -------------------------------
l_sh, r_sh = joint_2d["left_shoulder"], joint_2d["right_shoulder"]
l_hip, r_hip = joint_2d["left_hip"], joint_2d["right_hip"]

torso_width  = abs(r_sh[0] - l_sh[0])
torso_height = abs(((l_hip[1] + r_hip[1]) // 2) - l_sh[1])

sh_h, sh_w = shirt_img.shape[:2]
scale_sh = min(torso_width / sh_w, torso_height / sh_h) * 1.5
resized_shirt = cv2.resize(shirt_img, (int(sh_w*scale_sh), int(sh_h*scale_sh)), interpolation=cv2.INTER_AREA)

offset_x = int((l_sh[0] + r_sh[0]) // 2 - resized_shirt.shape[1] // 2)
offset_y = l_sh[1]
overlay = overlay_rgba(smpl_img, resized_shirt, offset_x, offset_y)

# -------------------------------
# 6. Auto-fit pants
# -------------------------------
pant_width  = abs(r_hip[0] - l_hip[0])
pant_height = abs(((joint_2d["left_knee"][1] + joint_2d["right_knee"][1]) // 2) - l_hip[1])

pant_h, pant_w = pant_img.shape[:2]
scale_pant_w = pant_width / pant_w * 1.3
scale_pant_h = pant_height / pant_h * 1.2
new_w_p = int(pant_w * scale_pant_w)
new_h_p = int(pant_h * scale_pant_h)
resized_pant = cv2.resize(pant_img, (new_w_p, new_h_p), interpolation=cv2.INTER_AREA)

offset_x_p = int((l_hip[0] + r_hip[0]) // 2 - resized_pant.shape[1] // 2)
offset_y_p = l_hip[1]
overlay = overlay_rgba(overlay, resized_pant, offset_x_p, offset_y_p)

# -------------------------------
# 7. Draw joint markers
# -------------------------------
for name, (u, v) in joint_2d.items():
    cv2.circle(overlay, (u, v), 5, (0, 0, 255), -1)   # red dot
    cv2.putText(overlay, name, (u+5, v-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1, cv2.LINE_AA)

# -------------------------------
# 8. Show result
# -------------------------------
cv2.imshow("SMPL + Auto Shirt + Pants + Joints", overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import cv2
import mediapipe as mp
import numpy as np

# -------------------------------
# Load clothes (with transparency)
# -------------------------------
shirt_img = cv2.imread("./database/shirt2.png", cv2.IMREAD_UNCHANGED)
pant_img  = cv2.imread("./database/pant.png", cv2.IMREAD_UNCHANGED)

if shirt_img is None or pant_img is None:
    raise FileNotFoundError("Shirt or pant image not found")

# -------------------------------
# Helper: Alpha overlay
# -------------------------------
def overlay_rgba(background, overlay_img, x, y):
    bh, bw = background.shape[:2]
    oh, ow = overlay_img.shape[:2]

    # ✅ Handle negative offsets
    if x < 0:
        overlay_img = overlay_img[:, -x:]
        ow = overlay_img.shape[1]
        x = 0
    if y < 0:
        overlay_img = overlay_img[-y:, :]
        oh = overlay_img.shape[0]
        y = 0

    if x >= bw or y >= bh:
        return background

    # Clip overlay if it goes outside frame
    if x + ow > bw:
        ow = bw - x
        overlay_img = overlay_img[:, :ow]
    if y + oh > bh:
        oh = bh - y
        overlay_img = overlay_img[:oh]

    if ow <= 0 or oh <= 0:
        return background

    overlay_img = overlay_img.astype(float)
    background_crop = background[y:y+oh, x:x+ow].astype(float)

    if overlay_img.shape[2] == 4:
        alpha = overlay_img[:, :, 3] / 255.0
        alpha = np.stack([alpha]*3, axis=-1)
    else:
        alpha = np.ones_like(overlay_img[:, :, :3])

    blended = alpha * overlay_img[:, :, :3] + (1 - alpha) * background_crop
    background[y:y+oh, x:x+ow] = blended.astype(np.uint8)
    return background

# -------------------------------
# Initialize MediaPipe Pose
# -------------------------------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

# -------------------------------
# Webcam Loop
# -------------------------------
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        lm = results.pose_landmarks.landmark

        # Get key joints
        l_shoulder = (int(lm[11].x * w), int(lm[11].y * h))
        r_shoulder = (int(lm[12].x * w), int(lm[12].y * h))
        l_hip      = (int(lm[23].x * w), int(lm[23].y * h))
        r_hip      = (int(lm[24].x * w), int(lm[24].y * h))
        l_knee     = (int(lm[25].x * w), int(lm[25].y * h))
        r_knee     = (int(lm[26].x * w), int(lm[26].y * h))
        l_ankle    = (int(lm[27].x * w), int(lm[27].y * h))
        r_ankle    = (int(lm[28].x * w), int(lm[28].y * h))

        # ---- SHIRT ----
        torso_width  = abs(r_shoulder[0] - l_shoulder[0])
        torso_height = abs(((l_hip[1] + r_hip[1]) // 2) - ((l_shoulder[1] + r_shoulder[1]) // 2))

        sh_h, sh_w = shirt_img.shape[:2]
        # ✅ Auto scaling: match detected torso + 15% margin
        target_sh_w = int(torso_width * 1.15)
        target_sh_h = int(torso_height * 1.15)

        if target_sh_w > 0 and target_sh_h > 0:
            resized_shirt = cv2.resize(shirt_img, (target_sh_w, target_sh_h))
            offset_x = int((l_shoulder[0] + r_shoulder[0]) // 2 - resized_shirt.shape[1] // 2)
            offset_y = min(l_shoulder[1], r_shoulder[1]) - int(resized_shirt.shape[0]*0.1)
            frame = overlay_rgba(frame, resized_shirt, offset_x, offset_y)

        # ---- PANTS ---- (modified to use ankles instead of knees)
        pant_width  = abs(r_hip[0] - l_hip[0])
        pant_height = abs(((l_ankle[1] + r_ankle[1]) // 2) - ((l_hip[1] + r_hip[1]) // 2))

        p_h, p_w = pant_img.shape[:2]
        # ✅ Auto scaling: match detected lower body (hip to ankle) + 10% margin
        target_p_w = int(pant_width * 1.10)
        target_p_h = int(pant_height * 1.20)

        if target_p_w > 0 and target_p_h > 0:
            resized_pant = cv2.resize(pant_img, (target_p_w, target_p_h))
            offset_x_p = int((l_hip[0] + r_hip[0]) // 2 - resized_pant.shape[1] // 2)
            offset_y_p = min(l_hip[1], r_hip[1]) - int(resized_pant.shape[0]*0.05)
            frame = overlay_rgba(frame, resized_pant, offset_x_p, offset_y_p)

        # Debug markers (hips, knees, ankles etc.)
        for pt in [l_shoulder, r_shoulder, l_hip, r_hip, l_knee, r_knee, l_ankle, r_ankle]:
            cv2.circle(frame, pt, 5, (0,0,255), -1)

    cv2.imshow("Realtime Virtual Try-On", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
pose.close()
cv2.destroyAllWindows()


In [13]:
import cv2
import mediapipe as mp
import numpy as np

# -------------------------------
# User Input: Select Two Clothes
# -------------------------------
top_type = "shirt"    # choose from: "shirt", "fullsleeve"
top_path = "./database/fullsleeve.png"  

bottom_type = "pant"  # choose from: "pant", "skirt"
bottom_path = "./database/pant.png"

top_img = cv2.imread(top_path, cv2.IMREAD_UNCHANGED)
bottom_img = cv2.imread(bottom_path, cv2.IMREAD_UNCHANGED)

if top_img is None:
    raise FileNotFoundError(f"❌ Top cloth not found at {top_path}")
if bottom_img is None:
    raise FileNotFoundError(f"❌ Bottom cloth not found at {bottom_path}")

# -------------------------------
# Mediapipe Pose
# -------------------------------
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

landmark_names = {
    "l_shoulder": 11, "r_shoulder": 12,
    "l_hip": 23, "r_hip": 24,
    "l_ankle": 27, "r_ankle": 28,
    "l_wrist": 15, "r_wrist": 16,
    "l_toe": 31, "r_toe": 32
}

# -------------------------------
# Helper: overlay transparent image
# -------------------------------
def overlay_transparent(background, overlay, x, y):
    h, w = overlay.shape[:2]
    if x >= background.shape[1] or y >= background.shape[0]:
        return background
    if x < 0:
        overlay = overlay[:, -x:]; w = overlay.shape[1]; x = 0
    if y < 0:
        overlay = overlay[-y:, :]; h = overlay.shape[0]; y = 0
    if x + w > background.shape[1]:
        w = background.shape[1] - x; overlay = overlay[:, :w]
    if y + h > background.shape[0]:
        h = background.shape[0] - y; overlay = overlay[:h, :]
    if overlay.shape[2] < 4:
        return background

    overlay_img = overlay[:, :, :3]
    mask = overlay[:, :, 3:] / 255.0
    roi = background[y:y+h, x:x+w]
    if roi.shape[:2] != overlay_img.shape[:2]:
        return background
    blended = (1.0 - mask) * roi + mask * overlay_img
    background[y:y+h, x:x+w] = blended.astype(np.uint8)
    return background

# -------------------------------
# Cloth Placement
# -------------------------------
def place_cloth(frame, cloth_img, cloth_type, get_point):
    if cloth_img is None:
        return frame

    if cloth_type == "pant":
        # Hips → Toes
        l_hip, r_hip = get_point("l_hip"), get_point("r_hip")
        l_toe, r_toe = get_point("l_toe"), get_point("r_toe")
        hip_w = np.linalg.norm(np.array(r_hip) - np.array(l_hip))
        leg_h = np.linalg.norm(np.array(l_toe) - np.array(l_hip))
        new_w = int(hip_w * 2.2)
        new_h = int(leg_h * 1.05)
        resized = cv2.resize(cloth_img, (new_w, new_h))
        cx = int((l_hip[0] + r_hip[0]) / 2 - new_w / 2)
        cy = int(min(l_hip[1], r_hip[1]))
        return overlay_transparent(frame, resized, cx, cy)

    elif cloth_type == "skirt":
        # Only Hips
        l_hip, r_hip = get_point("l_hip"), get_point("r_hip")
        hip_w = np.linalg.norm(np.array(r_hip) - np.array(l_hip))
        new_w = int(hip_w * 2.2)
        new_h = int(hip_w * 2.0)  # approximate skirt length
        resized = cv2.resize(cloth_img, (new_w, new_h))
        cx = int((l_hip[0] + r_hip[0]) / 2 - new_w / 2)
        cy = int(min(l_hip[1], r_hip[1]))
        return overlay_transparent(frame, resized, cx, cy)

    elif cloth_type == "shirt":
        # Only Shoulders
        l_sh, r_sh = get_point("l_shoulder"), get_point("r_shoulder")
        shoulder_w = np.linalg.norm(np.array(r_sh) - np.array(l_sh))
        new_w = int(shoulder_w * 2.0)
        new_h = int(shoulder_w * 2.2)  # torso length approx
        resized = cv2.resize(cloth_img, (new_w, new_h))
        cx = int((l_sh[0] + r_sh[0]) / 2 - new_w / 2)
        cy = int(min(l_sh[1], r_sh[1])) - int(new_h * 0.1)
        return overlay_transparent(frame, resized, cx, cy)

    elif cloth_type == "fullsleeve":
        # Same logic as shirt (overlay properly on bottom)
        l_sh, r_sh = get_point("l_shoulder"), get_point("r_shoulder")
        shoulder_w = np.linalg.norm(np.array(r_sh) - np.array(l_sh))
        new_w = int(shoulder_w * 2.0)
        new_h = int(shoulder_w * 2.2)  # torso length approx
        resized = cv2.resize(cloth_img, (new_w, new_h))
        cx = int((l_sh[0] + r_sh[0]) / 2 - new_w / 2)
        cy = int(min(l_sh[1], r_sh[1])) - int(new_h * 0.1)
        return overlay_transparent(frame, resized, cx, cy)
    return frame

# -------------------------------
# Webcam
# -------------------------------
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        lm = results.pose_landmarks.landmark
        def get_point(name):
            idx = landmark_names[name]
            return int(lm[idx].x * w), int(lm[idx].y * h)

        # Place bottom first (pant/skirt), then top (shirt/full sleeve)
        frame = place_cloth(frame, bottom_img, bottom_type, get_point)
        frame = place_cloth(frame, top_img, top_type, get_point)

    cv2.imshow("Virtual Try-On", frame)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
cv2.destroyAllWindows()
